Retrievers - 벡터 저장소에서 문서를 검색하는 도구   
Langchain은 다양한 검색 알고리즘을 지원함


In [2]:
# Load data -> Text split

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

loader = PyMuPDFLoader('300720_한일시멘트_2023.pdf') # 파일 읽고
data = loader.load() # 로드
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, # 1000개씩 끊고
    chunk_overlap=200, # 200개는 겹치게
    encoding_name='cl100k_base'
)

documents = text_splitter.split_documents(data)
len(documents)

/Users/dusanbaek/.pyenv/versions/3.11.9/lib/python3.11/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 78 of document 300720_한일시멘트_2023.pdf
  warnings.warn(


229

In [3]:
# 벡터스토어에 문서 임베딩을 저장
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

# documents에 대하여 한국어 임베딩 모델을 사용하여 cosine 유사도를 측정 기준으로 사용하여 vector store 생성
vectorstore = FAISS.from_documents(documents,
                                   embedding = embeddings_model,
                                   distance_strategy = DistanceStrategy.COSINE  
                                  )

/var/folders/9y/8gn45jd13x7_67gf0f254lg40000gn/T/ipykernel_4701/2091906221.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(


In [5]:
# 검색 쿼리
query = '한일시멘트에 대해 알려줘.'

# 가장 유사도가 높은 문장을 하나만 추출
retriever = vectorstore.as_retriever(search_kwargs={'k': 1})

docs = retriever.get_relevant_documents(query)
print(len(docs))
docs[0].page_content

1


'MATERIALITY ASSESSMENT\n중대이슈 결정 프로세스\nSTEP 1\n한일의 ESG경영  \n환경 분석\n한일을 둘러싼 지속가능경영 이슈들은 주변 상황과 맞물리며 부정 및 긍정 영향, 단기 및 장기 영향, \n가역적 및 불가역적 영향, 의도한 영향과 의도치 않은 영향 등 다양한 방향에서 실제 및 잠재적 영\n향을 미치게 됩니다. 한일은 지속가능경영 이슈의 영향을 파악하기 위해 글로벌 이니셔티브와 ESG\n정보공시가이드에서 요구하는 ESG경영 요소, 투자자 요구사항, 정책 및 규제 변화 등을 분석하여 \nESG 이슈 Pool을 구성하고 미디어 분석과 동종사 벤치마킹을 통해 한일에 요구되는 주요 중대이슈\n를 도출하였습니다. 이 과정을 통해 총 48개 이슈 Pool을 도출하였고 그 중 한일의 ESG경영과 관련\n성이 높은 20개 주요 이슈를 선정하였습니다. \nSTEP 2\n실제 및 잠재적  \n임팩트 식별\n도출된 ESG이슈가 미치는 영향의 우선순위를 지정하기 위해 사업연관성(Relevance)과 이해관계\n자 영향(Impact)에 대한 중대성 평가가 이루어졌습니다. 사업연관성은 이슈별로 한일이 속해 있는 \n산업의 ESG연관성을 분석하였고 이해관계자 영향은 2023년 4월 5일부터 2023년 4월 7일까지 이\n해관계자 설문을 실시하여 각 이슈별 이해관계자 관심도를 분석하였습니다. \n중대성 평가 결과 한일이 우선적으로 대응해야 할 5개 핵심이슈를 도출하였으며, 중대성평가 진행\n과정과 도출된 핵심 이슈는 경영진 및 이사회 보고를 통해 유효성 검토를 마쳤습니다. 2022년 선정\n된 핵심이슈는 기후변화 대응 전략, 오염물질 관리, 안전하고 건강한 작업환경 구축, 친환경 사업 개\n발, 소비자 만족 및 제품 품질이며, 본 보고서를 통해 이를 중심으로 하는 주요 활동 성과와 향후 계\n획을 투명하게 보고하고자 합니다.\nSTEP 3\n영향의 중대성 평가\nSTEP 4\n우선순위 유효성 검토 및  \n핵심 이슈 도출'

In [7]:
# MMR - 다양성 고려 (lambda_mult = 0.5)
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'fetch_k': 50, 'lambda_mult': 0.5}, # 50 개의 후보 중에서 5개를 반환한다는 소리; lambda_mult를 0.5로 하면 유사도/다양성 반반 사용
)

docs = retriever.get_relevant_documents(query)
print(len(docs))
docs[0].page_content

5


'MATERIALITY ASSESSMENT\n중대이슈 결정 프로세스\nSTEP 1\n한일의 ESG경영  \n환경 분석\n한일을 둘러싼 지속가능경영 이슈들은 주변 상황과 맞물리며 부정 및 긍정 영향, 단기 및 장기 영향, \n가역적 및 불가역적 영향, 의도한 영향과 의도치 않은 영향 등 다양한 방향에서 실제 및 잠재적 영\n향을 미치게 됩니다. 한일은 지속가능경영 이슈의 영향을 파악하기 위해 글로벌 이니셔티브와 ESG\n정보공시가이드에서 요구하는 ESG경영 요소, 투자자 요구사항, 정책 및 규제 변화 등을 분석하여 \nESG 이슈 Pool을 구성하고 미디어 분석과 동종사 벤치마킹을 통해 한일에 요구되는 주요 중대이슈\n를 도출하였습니다. 이 과정을 통해 총 48개 이슈 Pool을 도출하였고 그 중 한일의 ESG경영과 관련\n성이 높은 20개 주요 이슈를 선정하였습니다. \nSTEP 2\n실제 및 잠재적  \n임팩트 식별\n도출된 ESG이슈가 미치는 영향의 우선순위를 지정하기 위해 사업연관성(Relevance)과 이해관계\n자 영향(Impact)에 대한 중대성 평가가 이루어졌습니다. 사업연관성은 이슈별로 한일이 속해 있는 \n산업의 ESG연관성을 분석하였고 이해관계자 영향은 2023년 4월 5일부터 2023년 4월 7일까지 이\n해관계자 설문을 실시하여 각 이슈별 이해관계자 관심도를 분석하였습니다. \n중대성 평가 결과 한일이 우선적으로 대응해야 할 5개 핵심이슈를 도출하였으며, 중대성평가 진행\n과정과 도출된 핵심 이슈는 경영진 및 이사회 보고를 통해 유효성 검토를 마쳤습니다. 2022년 선정\n된 핵심이슈는 기후변화 대응 전략, 오염물질 관리, 안전하고 건강한 작업환경 구축, 친환경 사업 개\n발, 소비자 만족 및 제품 품질이며, 본 보고서를 통해 이를 중심으로 하는 주요 활동 성과와 향후 계\n획을 투명하게 보고하고자 합니다.\nSTEP 3\n영향의 중대성 평가\nSTEP 4\n우선순위 유효성 검토 및  \n핵심 이슈 도출'

In [8]:
# MMR - 다양성 고려 (lambda_mult 작을수록 더 다양하게 추출)
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15} # lambda가 0에 가까울 수록 다양성을 우선하게 된다!
)

docs = retriever.get_relevant_documents(query)
print(len(docs))
docs[-1].page_content

5


'한일 인권경영 선언문\n한일은 인간을 존중하고, 미래환경을 생각하는 환경친화기업으로 이\n해관계자의 인권 보호와 증진을 위해 2022년 인권경영 선언문을 채\n택하였습니다. 한일은 세계인권선언(Universal Declaration of Hu-\nman Rights), 국제노동기구 헌장(ILO Constitution) 등 인권, 노동\n관련 국제 규범을 준수하고 있으며, 글로벌 ESG 스탠더드에서 요구\n하는 모든 아동노동과 강제노동 금지 원칙을 준수하고 있습니다. 인\n권경영 선언문은 한일의 모든 임직원과 고객에게 동등하게 적용되며, \n차별금지, 안전한 근로환경과 노동 3권 보장, 직장 내 괴롭힘 방지, 고\n객 보호, 개인 정보보호, 인권침해 예방과 구제를 인권 경영 키워드로 \n선정하고 있습니다. 한일은 인권경영 선언문을 바탕으로 세부 주제별 \n인권경영 내부 지침서를 제정할 계획이며, 협력사를 비롯한 모든 이\n해관계자에게 인권 정책을 존중할 것을 권고하고 있습니다. \n우덕재단 사회공헌 활동 규모 \n372억 원 (1983~2022 누적)\n2\n상생협력형 내일채움 공제 제도 \n한일 지원금 1억 3,494만원 지급\n3\n95\nHANIL Sustainability Report\n94\nHANIL Sustainability Report\n한일 임직원 평균 근속연수 \n19.2년\n1\n차별금지\n안전, 환경\n노동3권 보장\n직장 내 괴롭힘 \n방지\n고용평등\n고객 보호\n개인정보보호\n인권침해 예방, \n구제\n> 인권경영 로드맵\n4단계\n3단계\n2단계\n1단계\n인권영향 평가 실시\n•  \x07인권영향평가 실시계획 수립\n•  \x07인권경영 가이드라인 및  \n \n체크리스트 교육\n•  \x07인권경영위원회 평가 자료 제출\n•  \x07인권경영위원회 평가 및  \n \n결과 보고서 작성\n•  \x07최고 경영진에 보고 및 공개\n인권경영 체계 구축\n•  \x07인권경영 추진 시스템 구축\n•  \x07인권경영 선언 및 공표\n

In [10]:
# Similarity score threshold (기준 스코어 이상인 문서를 대상으로 추출)
retriever = vectorstore.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.3} # threshold가 0.3 이상으로 설정. 유사도가 그보다 떨어지면 필터링됨.
)

docs = retriever.get_relevant_documents(query)
print(len(docs))

/Users/dusanbaek/.pyenv/versions/3.11.9/lib/python3.11/site-packages/langchain_core/vectorstores/base.py:1086: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='cb794e34-e4f5-4b69-8091-a61306d37a17', metadata={'source': '300720_한일시멘트_2023.pdf', 'file_path': '300720_한일시멘트_2023.pdf', 'page': 16, 'total_pages': 79, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'producer': 'iLovePDF', 'creationDate': "D:20230728104736+09'00'", 'modDate': "D:20230728111840+09'00'", 'trapped': ''}, page_content='MATERIALITY ASSESSMENT\n중대이슈 결정 프로세스\nSTEP 1\n한일의 ESG경영  \n환경 분석\n한일을 둘러싼 지속가능경영 이슈들은 주변 상황과 맞물리며 부정 및 긍정 영향, 단기 및 장기 영향, \n가역적 및 불가역적 영향, 의도한 영향과 의도치 않은 영향 등 다양한 방향에서 실제 및 잠재적 영\n향을 미치게 됩니다. 한일은 지속가능경영 이슈의 영향을 파악하기 위해 글로벌 이니셔티브와 ESG\n정보공시가이드에서 요구하는 ESG경영 요소, 투자자 요구사항, 정책 및 규제 변화 등을 분석하여 \nESG 이슈 Pool을 구성하고 미디어 분석과 동종사 벤치마킹을 통해 한일에 요구되는 주요 중대이슈\n를 도출하였습니다. 이 과정을 통해 총 48개 이슈 Pool을 도출하였고 그 중 한일의 ESG경영

0


In [ ]:
# 문서 객체의 metadata를 이용한 필터링
retriever = vectorstore.as_retriever(
    search_kwargs={'filter': {'format':'PDF 1.6'}} # 그냥 pdf 1.4버전만 쓰겠다는 소리.맥의 경우, 정보 가져오기를 누르고 추가 정보를 통해 pdf의 버전을 볼 수 있음.
)

docs = retriever.get_relevant_documents(query)
print(len(docs))
docs[0]

4


Document(id='cb794e34-e4f5-4b69-8091-a61306d37a17', metadata={'source': '300720_한일시멘트_2023.pdf', 'file_path': '300720_한일시멘트_2023.pdf', 'page': 16, 'total_pages': 79, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 16.4 (Macintosh)', 'producer': 'iLovePDF', 'creationDate': "D:20230728104736+09'00'", 'modDate': "D:20230728111840+09'00'", 'trapped': ''}, page_content='MATERIALITY ASSESSMENT\n중대이슈 결정 프로세스\nSTEP 1\n한일의 ESG경영  \n환경 분석\n한일을 둘러싼 지속가능경영 이슈들은 주변 상황과 맞물리며 부정 및 긍정 영향, 단기 및 장기 영향, \n가역적 및 불가역적 영향, 의도한 영향과 의도치 않은 영향 등 다양한 방향에서 실제 및 잠재적 영\n향을 미치게 됩니다. 한일은 지속가능경영 이슈의 영향을 파악하기 위해 글로벌 이니셔티브와 ESG\n정보공시가이드에서 요구하는 ESG경영 요소, 투자자 요구사항, 정책 및 규제 변화 등을 분석하여 \nESG 이슈 Pool을 구성하고 미디어 분석과 동종사 벤치마킹을 통해 한일에 요구되는 주요 중대이슈\n를 도출하였습니다. 이 과정을 통해 총 48개 이슈 Pool을 도출하였고 그 중 한일의 ESG경영과 관련\n성이 높은 20개 주요 이슈를 선정하였습니다. \nSTEP 2\n실제 및 잠재적  \n임팩트 식별\n도출된 ESG이슈가 미치는 영향의 우선순위를 지정하기 위해 사업연관성(Relevance)과 이해관계\n자 영향(Impact)에 대한 중대성 평가가 이루어졌습니다. 사업연관성은 이슈별로 한일이 속해 

In [13]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# Retrieval
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15}
)

docs = retriever.get_relevant_documents(query)

# Prompt: {context} 맥락에 따라 {question}에 맞는 응답을 생성하시오.
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# Model
llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0, # 창의성 0
    max_tokens=500,
)


def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs]) # 상위 5개에 해당하는 page_content를 띄어쓰기로 모두 더해서 string 데이터로 만들어 준 후에

# Chain
chain = prompt | llm | StrOutputParser() # 이걸 넣어서 응답을 생성하는 chain을 생성

# Run
response = chain.invoke({'context': (format_docs(docs)), 'question':query})
response

"한일시멘트는 단양, 영월에 3개의 포틀랜드 시멘트 생산 공장과 평택, 당진, 포항에 3개의 슬래그 시멘트 생산 공장을 운영하고 있으며, 전국적인 유통망을 확보하여 시멘트 및 콘크리트 원·부재료를 판매하고 있습니다. 한일시멘트는 고객사를 7개 권역으로 구분하고, 신제품 개발 및 품질 관리를 위해 '찾아가는 비포서비스'를 제공하고 있습니다. 이 서비스는 라돈 측정, 콘크리트 단열 온도 상승 실험 등 고가의 장비가 필요한 데이터 분석 서비스와 기술 지원을 포함합니다. 또한, 한일은 인권경영 선언문을 채택하여 인권 보호와 증진을 위해 노력하고 있으며, 차별금지, 안전한 근로환경, 고객 보호 등의 원칙을 준수하고 있습니다."

In [14]:
# 멀티 쿼리 생성
from langchain.retrievers.multi_query import MultiQueryRetriever

question = '한일시멘트의 최근 영업 실적을 알려줘.'

llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
    max_tokens=500,
)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['한일시멘트의 최근 매출 및 영업 성과에 대한 정보를 제공해줄 수 있나요?  ', '한일시멘트의 최신 영업 실적과 관련된 데이터가 궁금합니다.  ', '한일시멘트의 최근 재무 성과와 영업 결과를 확인할 수 있을까요?']


5

In [15]:
unique_docs[0].page_content

'십을 구축하여 사회적 가치 창출에도 기여하겠습니다.\n이해관계자 여러분께 한일의 지속가능경영에 대한 의지와 추진 과정을 알리고 이해\n를 돕기 위해 최초로 그룹 관점의 지속가능경영보고서를 발간하게 되었습니다. 지속\n가능한 가치를 창출하는 한일의 긴 여정에 이해관계자 여러분들의 변함없는 기대와 \n관심을 부탁드립니다. \n감사합니다.\n한일홀딩스   \n대표이사  박지훈\n5\nHANIL Sustainability Report\n4\nHANIL Sustainability Report'

In [16]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


# Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# Model
llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
)

# 검색된 문서들을 하나의 문자열로 포맷팅
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# Chain
chain = (
    {'context': retriever_from_llm | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Run
response = chain.invoke('한일시멘트의 최근 영업실적을 요약해서 알려주세요.')
response

INFO:langchain.retrievers.multi_query:Generated queries: ['한일시멘트의 최근 매출 및 영업 성과에 대한 정보를 제공해 주세요.  ', '한일시멘트의 최신 영업 실적에 대한 요약을 부탁드립니다.  ', '한일시멘트의 최근 재무 성과와 영업 실적을 정리해 주세요.']


'한일시멘트는 최근 시멘트 업계에서 유연탄 등 글로벌 원자재 가격 폭등으로 인해 수익성이 악화되었으며, 화물연대 파업과 건설경기 침체 등으로 어려운 시기를 겪었습니다. 이러한 대내외적인 어려움에도 불구하고, 한일시멘트는 장기적인 안목과 적극적인 투자를 통해 기후변화를 기회로 전환하고 지속 가능한 성장을 추구하고 있습니다.'

In [17]:
# 기본 검색기

question = '한일시멘트의 최근 영업실적을 알려줘.'

llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
    max_tokens=500,
)

base_retriever = vectorstore.as_retriever(
                                search_type='mmr',
                                search_kwargs={'k':7, 'fetch_k': 20})

docs = base_retriever.get_relevant_documents(question)
print(len(docs))

7


In [18]:
docs[0].page_content

'CEO MESSAGE\n한일시멘트 · 한일현대시멘트\n장기적인 안목과 적극적인 투자로 \n기후변화를 위기가 아닌 기회로 전환시키는 \n한일시멘트. 한일현대시멘트가 되겠습니다. \n존경하는 한일 이해관계자 여러분, 안녕하십니까.\n한일의 지속적인 성장과 발전에 변함없는 성원과 관심을 기울여 주신 주주, 고객,  \n협력사, 임직원분들께 감사의 인사를 드립니다.\n지난해 시멘트 업계는 유연탄 등 글로벌 원자재 가격 폭등으로 수익성이 악화되었고, \n화물연대 파업, 건설경기 침체 등 대내외적으로 어려운 시기를 보내야 했습니다. 이에 \n더해 폭염 · 폭우 · 가뭄 등 전 세계적인 기상 이변은 자연재해뿐만 아니라 생태계 파괴, \n식량위기 등 다양한 문제들을 야기하고 있습니다. 친환경의 시대를 넘어 인류 생존을 \n위한 필(必)환경의 시대임을 절감케하는 변화가 현실이 된 것입니다.\n한일은 급변하는 대내외 환경 속에서도 장기적인 안목을 가지고 환경 · 사회 · 지배\n구조를 중심으로 지속가능경영을 추구하는 ESG 실천에 역량을 집중하고 있습니다. \n특히 온실가스 및 대기오염물질 감축과 같은 환경 이슈는 위기를 넘어 기회로 전환\n할 수 있도록 적극적인 대응에 나서고 있습니다. \n한일은 ‘2030 온실가스 30% 감축, 2050 Net Zero’ 달성을 위해서 저탄소 친환경 \n생산공정 구축에 필요한 기술 개발 및 설비투자를 지속하고 있습니다. 2025년까지 \n단계적으로 시멘트 공장의 소성로를 개조하여 화석연료 의존도를 낮추고, 순환연료\n의 비중을 확대하는 한편, 석회석 등 천연원료를 순환원료로 대체하여 자원순환형 \n사회 구축에 기여하도록 하겠습니다. 또한 시멘트 소성공정 중에 발생되는 폐열을 \n회수하여 전기를 생산하는 ECO 발전을 확대하여 전력 자립도를 높이겠습니다.\n또한 환경을 둘러싼 규제가 강화되고 기업의 오염물질 관리가 더욱 중요해지는 사회\n변화에 발맞추어 엄격한 내부기준으로 오염물질 배출을 최소화하고, 생산활동의 환경\n영향을 지속적으로 관리

In [19]:
# 문서 압축기를 연결하여 구성

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=base_retriever
)

compressed_docs = compression_retriever.get_relevant_documents(question)
print(len(compressed_docs))

0


In [20]:
compressed_docs

[]